<a href="https://colab.research.google.com/github/englconti/ImersaoAlura-Maio-2025-AI/blob/main/Copy_of_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Para saber quando será a próxima Imersão IA com Google Gemini da Alura, a melhor forma é verificar diretamente no site da Alura:

*   **Site da Alura:** Procure na página inicial, na seção de cursos de Inteligência Artificial ou use a ferramenta de busca do site por "Imersão IA Google Gemini".

A Alura costuma divulgar as datas e horários das imersões com antecedência em seu site e redes sociais.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google #####################
    config = { "tools": [{ "google_search": {}}] }
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As aulas serão online e gratuitas e, ao final do curso, você receberá um certificado da Alura e do Google. As inscrições podem ser feitas até o dia 11 de maio.

A Imersão IA é voltada para todas as pessoas interessadas em tecnologia e que querem usar IA para otimizar o trabalho e transformar ideias em soluções reais. Não é necessário ter conhecimento prévio para participar. Durante o curso, você aprenderá a criar prompts poderosos, automatizar tarefas e desenvolver um sistema de agentes autônomos que trabalham sozinhos. Você também aprenderá os fundamentos da engenharia de prompts e a criar um chatbot com a API Gemini.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk



In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()

    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Atue como um assistente de pesquisa. Sua tarefa e usar a ferramenta de busca do google (google_search) para recuperar as ultimas noticias de lancamentos muito relevantes sobre o topico abaixo.
        Foque em no maximo 5 lancamentos relevantes, com base na quantidade e entusiasmo das noticias sobre ele.
        Se um tema tiver poucas noticias ou poucas reacoes entusiasmadas, e possivel que ele nao seja tao relevante assim e pode ser substituido por outro que tenha mais.
        Esses lancamentos relevantes devem ser atuais, de no maximo um mes antes da data de hoje.
        """,
        description="Agente para buscar informacoes no google",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)

    return lancamentos


In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Voce e um planejador de conteudo e especialista em redes sociais.
        Com base na lista de lancamentos mais recentes e relevantes buscador, voce deve:
        Usar a ferramente de busca do Google (google_search) para criar um plano sobre quais sao os pontos mais relevantes que poderiamos abordar em um post sobre cada um deles.
        Voce tambem pode usar o (google_search) para encontrar mais informacoes sobre os temas e aprofundar.
        Ao final, voce ira escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)

    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    rascunho = call_agent(redator, entrada_do_agente_redator)

    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)

    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Voce nao digitou o topico!!")
else:
  print(f"Maravilha! Vamos fazer o trabalho do sobre o topico {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("] === Resultados buscados ===>")
  display(to_markdown(lancamentos_buscados))
  # print(to_markdown(lancamentos_buscados))

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("] === Plano ===>")
  display(to_markdown(plano_de_post))
  print("==============================")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("] === Rascunho ===>")
  display(to_markdown(rascunho_de_post))
  print("==============================")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("] === Proposta final ===>")
  display(to_markdown(post_final))
  print("==============================")





🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: FORRÓ
Maravilha! Vamos fazer o trabalho do sobre o topico FORRÓ
] === Resultados buscados ===>


> Para encontrar os lançamentos mais relevantes de forró no último mês, vou realizar algumas buscas no Google sobre lançamentos recentes e notícias do gênero.
> 
> 
> Com base nas minhas buscas, aqui estão alguns dos lançamentos e eventos de forró mais relevantes e recentes (considerando a data de hoje, 16 de maio de 2025):
> 
> 1.  **Lançamentos Musicais:**
>  *   **Singles e Álbuns:** Há diversas playlists e seleções de forró sendo lançadas em maio de 2025. Algumas destacam músicas de artistas populares como Wesley Safadão, Natanzinho Lima, Mari Fernandez e Xand Avião.
>  *   **Colaborações:** Lucas Aboiador e Tarcísio do Acordeon lançaram o single colaborativo "Maior Riqueza". Zé Vaqueiro e Xand Avião uniram-se no single "Que Pancada de Mulher".
> 2.  **Festivais e Eventos:**
>  *   **Forró da Advocacia 2025:** O evento aconteceu em 10 de maio no Centro de Convenções de Sergipe, com shows de Aline Souza, Danielzinho JR, Kátia Cilene e Guilherme Ferri.
>  *   **Festivais Internacionais:** Vários festivais de forró estão programados para ocorrer em diversas cidades da Europa ao longo de maio e junho de 2025.
>  *   **Arraiá do Povo e Vila do Forró:** A Fundação de Cultura e Arte Aperipê de Sergipe (Funcap) divulgou os estabelecimentos selecionados para bares e restaurantes nestes eventos.
>  *   **Forró no Parque:** O evento teve edições em março e abril de 2025 em Salvador, Bahia.
>  *   **Forró Xperience 2025:** Festival que ocorrerá em Leisnig, Alemanha, entre 26 e 29 de junho de 2025.
> 3.  **Outros Destaques:**
>  *   **Concurso Rei e Rainha do País do Forró 2025:** O Governo de Sergipe abriu inscrições para o concurso, com a coroação dos vencedores marcada para 29 de maio.
>  *   **Planeta Forró:** O site continua a ser uma fonte de notícias sobre o mundo do forró, destacando lançamentos e eventos.
> 
> É importante notar que a popularidade de artistas como Wesley Safadão, Mari Fernandez, Nattan e Xand Avião continua em alta, com suas músicas frequentemente incluídas em playlists de "paredão" e seleções de maiores sucessos.
> 


] === Plano ===>


> Com base nas informações fornecidas, podemos criar um plano de conteúdo focado nos seguintes temas de forró:
> 
> 
> Com base nas informações e pesquisas, o tema mais relevante para um post de redes sociais é:
> 
> **Tema:** Festivais de Forró na Europa em 2025
> 
> **Pontos Relevantes:**
> 
> *   **Crescimento e Popularidade:** O aumento do número de festivais de forró na Europa demonstra o crescente interesse e a expansão da cultura do forró para além do Brasil.
> *   **Diversidade Geográfica:** Festivais acontecem em diversos países, como Alemanha, Portugal, Espanha, França, Suíça, entre outros, mostrando a ampla aceitação do ritmo.
> *   **Variedade de Festivais:** Há festivais de diferentes estilos e durações, desde fins de semana temáticos até eventos mais longos, oferecendo opções para todos os gostos.
> *   **Atrações:** Muitos festivais contam com a presença de renomados artistas brasileiros, como Mestrinho e Nonato Lima, além de professores de dança e músicos locais.
> *   **Impacto Cultural:** Os festivais promovem a cultura brasileira, especialmente a nordestina, no exterior, gerando intercâmbio cultural e fortalecendo a identidade do forró.
> 
> **Plano de Conteúdo:**
> 
> 1.  **Introdução:**
>     *   Começar com uma chamada chamativa sobre a expansão do forró na Europa e o grande número de festivais acontecendo em 2025.
>     *   Explicar brevemente o que é o forró e sua importância cultural no Brasil.
> 2.  **Destaque dos Principais Festivais:**
>     *   Mencionar alguns dos festivais mais populares e tradicionais, como o Forró Xperience (Alemanha), o Baião in Lisboa Festival (Portugal), e outros.
>     *   Incluir informações sobre datas, locais e atrações confirmadas, como a presença de artistas brasileiros renomados.
> 3.  **Impacto e Benefícios dos Festivais:**
>     *   Discutir como os festivais contribuem para a divulgação da cultura brasileira e o intercâmbio cultural entre Brasil e Europa.
>     *   Mencionar como os festivais geram oportunidades para artistas, professores de dança e músicos de forró.
> 4.  **Chamada para Ação:**
>     *   Incentivar os seguidores a conferirem a lista completa de festivais e planejarem suas viagens.
>     *   Criar um senso de comunidade, convidando os seguidores que já foram a festivais de forró na Europa a compartilharem suas experiências nos comentários.
> 5.  **Conteúdo Visual:**
>     *   Utilizar fotos e vídeos de edições anteriores dos festivais, mostrando a animação e a energia dos eventos.
>     *   Criar um design atraente e informativo para o post, com cores vibrantes e fontes legíveis.
> 
> Este plano de conteúdo visa engajar o público interessado em forró, tanto no Brasil quanto na Europa, e promover a cultura brasileira através da divulgação dos festivais europeus.
> 


] === Rascunho ===>


> ## Rascunho de Post para Instagram:
> 
> 🎶 Prepare o chapéu de couro e a fivela porque o forró tá dominando a Europa em 2025! 🎶
> 
> É isso mesmo, gente! O ritmo que embala o Brasil tá conquistando cada vez mais corações europeus, e a prova disso é a explosão de festivais de forró que vão rolar por lá no ano que vem. 💃🕺
> 
> Já marca na agenda: tem Forró Xperience na Alemanha, Baião in Lisboa Festival em Portugal e MUITO mais! 🌍 Prepare-se para dançar agarradinho ao som de Mestrinho, Nonato Lima e outros artistas incríveis que vão botar todo mundo pra xaxar!
> 
> E não é só música, viu? Esses festivais são um verdadeiro intercâmbio cultural, unindo brasileiros e europeus na paixão pelo forró, fortalecendo nossa cultura e criando momentos inesquecíveis. 🥰
> 
> E aí, já foi em algum festival de forró na Europa? Conta pra gente nos comentários qual foi a sua experiência! 👇 E se ainda não foi, corre pro site da Alura (link na bio 😉) pra conferir a lista completa dos festivais e planejar sua viagem!
> 
> Vem forrozear com a gente! 🔥
> 
> #ForróNaEuropa #Alura #CulturaBrasileira #FestivaisDeForró
> 


] === Proposta final ===>


> O rascunho está ótimo e pronto para publicar!
> 
